# asyncio
asyncio(Asynchronous I/O)는 비동기 프로그래밍을 위한 모듈이며 CPU 작업과 I/O를 병렬로 처리하게 해줍니다.

동기(synchronous) 처리는 특정 작업이 끝나면 다음 작업을 처리하는 순차처리 방식이고, 
비동기(asynchronous) 처리는 여러 작업을 처리하도록 예약한 뒤 작업이 끝나면 결과를 받는 방식입니다.


A. 코루틴

코루틴도 서브루틴(서브함수)처럼 기능들을 별도의 공간에 모아 놓고 있다는 점에서는 동일합니다. 
차이점이라 할 수 있는 것은, 서브루틴의 경우에는 메인루틴에서 특정 서브루틴의 공간으로 이동한 후에 리턴에 의해 호출자로 돌아와 
다시 프로세스를 진행하는데 반해 코루틴의 경우에는 루틴을 진행하는 중간에 멈추어서 특정 위치로 돌아갔다가 다시 원래 위치로 
돌아와 나머지 루틴을 수행할 수 있습니다. 
또 한가지 차이점은 서브루틴은 진입점과 반환점이 단 하나밖에 없어 메인루틴에 종속적이지만, 
코루틴은 진입지점이 여러개이기 때문에 메인루틴에 종속적이지 않아 대등하게 데이터를 주고 받을 수 있다는 특징이 있습니다. 
코루틴은 주로 동시성을 필요로 하는 UNITY 등의 게임프로그래밍에서 많이 사용하는 개념이라고 합니다.

In [ ]:
def test1(i):
    print('start test1 coroutine')
    while True:
        yield i
        i += 1

a = test1(5)
next(a)    # start test1 coroutine 출력 후 5출력, yield i 부분에서 멈춰있다.
next(a)    # 멈춰진 yield i 부분 다음줄의 5 += 1(i=6)을 수행한후 다음 6을 출력하고 yield i에서 멈춘다.
next(a)    # 7을 출력하고 yield i에서 멈춘다.
next(a)

위의 예제는 코루틴 객체를 생성한 후 next() 함수를 통해서 첫번째 yield 문에 도달합니다.
yield 문을 통해 메인루틴에 값을 전달한 후 코루틴 함수는 대기합니다. 다음 next()함수가 호출되면, 
멈춰져 있는 yield 부분 다음의 로직을 수행한 다음 한바퀴 돌아 yield 부분에서 또다시 멈추게 됩니다. 
아래의 예제에서는 next()를 무한정 호출할 수 있지만, 특정한 조건을 지정해주면 정해진 만큼만 next()를 호출할 수 있습니다.
더이상 호출할 수 없는 코루틴 함수를 next()로 호출하면 예외가 발생하게 됩니다.

In [ ]:
B. 네이티브 코루틴 만들기
asyncio를 사용하려면 다음과 같이 async def로 네이티브 코루틴을 만듭니다
(파이썬에서는 제너레이터 기반의 코루틴과 구분하기 위해 async def로 만든 코루틴은 네이티브 코루틴이라고 합니다)
async def 키워드는 파이썬 3.5 이상부터 사용 가능합니다.

In [ ]:
import asyncio
 
async def hello():    # async def로 네이티브 코루틴을 만듦
    print('Hello, world!')
 
loop = asyncio.get_event_loop()     # 이벤트 루프를 얻음
loop.run_until_complete(hello())    # hello가 끝날 때까지 기다림
loop.close()      

C. await로 네이티브 코루틴 실행하기
await 뒤에 코루틴 객체를 지정하면 해당 객체가 끝날 때까지 기다린 뒤 결과를 반환합니다. 
await 키워드는 파이썬 3.5 이상부터 사용 가능, 3.4에서는 yield from을 사용합니다.

변수 = await 코루틴객체

In [ ]:
import asyncio

async def add(a, b):
    print('add: {0} + {1}'.format(a, b))
    await asyncio.sleep(1.0)    # 1초 대기. asyncio.sleep도 네이티브 코루틴
    return a + b    # 두 수를 더한 결과 반환
 
async def print_add(a, b):
    result = await add(a, b)    # await로 다른 네이티브 코루틴 실행하고 반환값을 변수에 저장
    print('print_add: {0} + {1} = {2}'.format(a, b, result))
 
loop = asyncio.get_event_loop()             # 이벤트 루프를 얻음
loop.run_until_complete(print_add(1, 2))    # print_add가 끝날 때까지 이벤트 루프를 실행
loop.close()           # 이벤트 루프를 닫음

add함수는 1 + 2가 출력되고 1초 뒤에 print_add 함수 1 + 2 = 3이 출력됩니다.
print_add에서는 await로 add를 실행하고 반환값을 변수에 저장했습니다. 
이렇게 코루틴 안에서 다른 코루틴을 실행할 때는 await를 사용합니다.

add에서는 await asyncio.sleep(1.0)로 1초 대기한 뒤 return a + b로 두 수를 더한 결과를 반환합니다. 
사실 await asyncio.sleep(1.0)은 없어도 되지만 코루틴이 비동기로 실행되는 모습을 확인하기 위해 사용했습니다. 
특히 asyncio.sleep도 네이티브 코루틴이라 await를 사용해야 합니다.

D. 파이썬 3.4 이하에서 asyncio 사용하기

async def와 await는 파이썬 3.5에서 추가되었습니다. 
따라서 3.5 미만 버전에서는 사용할 수 없습니다. 파이썬 3.4에서는 다음과 같이 @asyncio.coroutine 데코레이터로 네이티브 코루틴을 만듭니다.
파이썬 3.3에서 asyncio는 pip install asyncio로 asyncio를 설치한 뒤 
@asyncio.coroutine 데코레이터와 yield from을 사용하면 됩니다. 단, 3.3 미만 버전에서는 asyncio를 지원하지 않습니다.